## GoogleMaps API: Get Pharmacies at Location

In [2]:
import requests as req
import json
import time
import ftfy
import geopandas as gpd


key = ''

place = 'Würzburg, Bayern, Deutschland'

### Get place IDs from Gmaps API and OSM and extract place geometry and bounds

OSM is used for precise location boundaries which cannot be retrieved from Gmaps.

In [3]:
# Google Maps API
gmaps_id_url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place}&inputtype=textquery&key={key}'
gmaps_id = req.get(gmaps_id_url).json()['candidates'][0]['place_id']

# OpenStreetMap
osm_id_url = f'https://nominatim.openstreetmap.org/search?q={place}&format=json&polygon_geojson=1&addressdetails=1'
osm_id_response = req.get(osm_id_url).json()
osm_id = osm_id_response[0]['osm_id']


# Get location geometry from Google Maps API
gmaps_geometry_url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={gmaps_id}&fields=geometry&key={key}'
gmaps_centroid_latlng = req.get(gmaps_geometry_url).json()['result']['geometry']['location']

# Extract the GeoJSON polygon from the OSM response and import it into a GeoDataFrame
bounds = gpd.read_file(json.dumps(osm_id_response[0]['geojson']), driver='GeoJSON')

In [10]:
print(json.dumps(gmaps_centroid_latlng, indent=2))

{
  "lat": 49.79130439999999,
  "lng": 9.9533548
}


In [9]:
print(json.dumps(osm_id_response, indent=2))

[
  {
    "place_id": 307671453,
    "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
    "osm_type": "relation",
    "osm_id": 62464,
    "boundingbox": [
      "49.7106843",
      "49.8454619",
      "9.8716285",
      "10.0144286"
    ],
    "lat": "49.79245",
    "lon": "9.932966",
    "display_name": "W\u00fcrzburg, Bayern, Deutschland",
    "class": "boundary",
    "type": "administrative",
    "importance": 0.9492862917438598,
    "icon": "https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png",
    "address": {
      "city": "W\u00fcrzburg",
      "state": "Bayern",
      "ISO3166-2-lvl4": "DE-BY",
      "country": "Deutschland",
      "country_code": "de"
    },
    "geojson": {
      "type": "Polygon",
      "coordinates": [
        [
          [
            9.8716285,
            49.7971542
          ],
          [
            9.8716715,
            49.7970188
          ],
          [
            9.872428

### Get all pharmacies within city bounds and filter for wrongly classificated results

In [4]:
# transform bounds gdf to UTM to calculate accurate distances
bounds_utm = bounds.to_crs(bounds.estimate_utm_crs())

# get a reference point at max-x and max-y of the location bounds and calculate radius (distance to centroid)
refpoint = gpd.GeoSeries(gpd.points_from_xy([bounds_utm.geometry.bounds.maxx[0]], [bounds_utm.geometry.bounds.maxy[0]]))
refpoint.crs = bounds_utm.crs

radius = bounds_utm.centroid.distance(refpoint)[0].round(0)

In [5]:
# Find pharmacies around the centroid of the location
find_pharmacies = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={gmaps_centroid_latlng["lat"]},{gmaps_centroid_latlng["lng"]}&radius={radius}&type=pharmacy&language=de&key={key}'
result = req.get(find_pharmacies).json()

found_pharmacies = result['results']


# Max 20 results are returned per request, so we need to use the next_page_token to get more results if available
if 'next_page_token' in result:
    
    while True:
        find_more_pharmacies = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={result["next_page_token"]}&key={key}'
        
        time.sleep(5)
        result = req.get(find_more_pharmacies).json()
        found_pharmacies.extend(result['results'])

        if 'next_page_token' not in result:
            break

In [6]:
# filter and prepare pharmacies
pharmacies = []

for pharmacy in found_pharmacies:

    # check if pharmacy is within bounds
    pharmacy_location = gpd.GeoSeries(gpd.points_from_xy([pharmacy['geometry']['location']['lng']], [pharmacy['geometry']['location']['lat']]))
    pharmacy_location.crs = bounds.crs

    if not bounds.contains(pharmacy_location)[0]:
        continue

    # check if pharmacy name contains "apotheke" or "pharmacy" to filter results wrongly classified as pharmacies
    if not any(x in pharmacy['name'].lower() for x in ['apotheke', 'pharmacy']):
        continue

    # extract location, name and id from results
    ph = {}
    ph['name'] = ftfy.fix_text(pharmacy['name'])
    ph['id'] = pharmacy['place_id']
    ph['location'] = pharmacy['geometry']['location']

    pharmacies.append(ph)

In [29]:
with open('pharmacies.json', 'r') as f:
    comp_pharms = json.load(f)

print(comp_pharms[0]["name"])

# compare pharmacies with existing pharmacies
"""for ph in pharmacies:
    if ph['id'] in comp_pharms:
        continue
    else:
        print(ph['name'])"""


Karmeliten-Apotheke


"for ph in pharmacies:\n    if ph['id'] in comp_pharms:\n        continue\n    else:\n        print(ph['name'])"

In [72]:
origin = "Kräuterwiese 20, 97273 Kürnach, Deutschland"
destination = "Frankenstraße 99, 97078 Würzburg, Deutschland"

# get origin and destination ids
origin_id = req.get(f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={origin}&inputtype=textquery&key={key}').json()['candidates'][0]['place_id']
destination_id = req.get(f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={destination}&inputtype=textquery&key={key}').json()['candidates'][0]['place_id']

route_data = {
    "origin": {
        "placeId": origin_id,
    },
    "destination": {
        "placeId": destination_id,
    },
    "computeAlternativeRoutes": "false",
    "travelMode": "DRIVE",
    "languageCode": "de-DE"
}

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": key,
    "X-Goog-FieldMask": "routes.duration,routes.distanceMeters"
}

url = "https://routes.googleapis.com/directions/v2:computeRoutes"

# calculate a simple route with the new Maps Routes API
route = req.post(url, json=route_data, headers=headers)

In [73]:
print(json.dumps(route.json(), indent=4))

{
    "routes": [
        {
            "distanceMeters": 7669,
            "duration": "482s"
        }
    ]
}


In [18]:
osm_geo_url = f'https://nominatim.openstreetmap.org/lookup?osm_ids=R62464&format=json&polygon_geojson=1'
osm_geo_response = req.get(osm_geo_url).json()

print(json.dumps(osm_geo_response, indent=2))

[
  {
    "place_id": 307671453,
    "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
    "osm_type": "relation",
    "osm_id": 62464,
    "boundingbox": [
      "49.7106843",
      "49.8454619",
      "9.8716285",
      "10.0144286"
    ],
    "lat": "49.79245",
    "lon": "9.932966",
    "display_name": "W\u00fcrzburg, Bayern, Deutschland",
    "class": "boundary",
    "type": "administrative",
    "importance": 0.6192862917438597,
    "address": {
      "city": "W\u00fcrzburg",
      "state": "Bayern",
      "ISO3166-2-lvl4": "DE-BY",
      "country": "Deutschland",
      "country_code": "de"
    },
    "geojson": {
      "type": "Polygon",
      "coordinates": [
        [
          [
            9.8716285,
            49.7971542
          ],
          [
            9.8716715,
            49.7970188
          ],
          [
            9.8724287,
            49.7946355
          ],
          [
            9.8729441,
            49.7946648
   